# Multivariate Regression

# Show the collections that have been generated

In [1]:
from pymongo import MongoClient
from process_collection import *
#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
collection = 'random_multivariate250'
client = MongoClient(mongoServer)
################################################

db = client[database]
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionNames = sorted(db.collection_names())
#collectionNames

# Formats parameters to be used for multivariate regression

In [2]:
import numpy as np
import statsmodels.api as sm
from get_cso import * 
from process_collection import * 
collectionName = collection #from cell above
ratio = 0.05
numSubcatch = 7
results = volume_reduction_and_runoff_volume_vs_lid_number(collectionName, numSubcatch, db)
csoResults = cso_reduction(collectionName, ratio, numSubcatch, results) 

### regression
y = csoResults["csoReduction"]
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results
reg_m(y,x)
print reg_m(y, x).summary()


[0.0, 50908990.797683954, 32760472.051656723, 46580343.76461649, 49410410.74556458, 45331302.63891637, 37784401.25248957, 55362989.65244472, 51530774.61229217, 52110339.120072246, 27742218.32134807, 39383598.038868666, 40430725.716780424, 42869750.75888419, 39802007.46013212, 50856861.29796028, 27582396.263497233, 31170027.473676324, 36954395.66757536, 53188691.984568596, 48324315.542868614, 29780021.9329921, 60344941.91529703, 36331871.2814883, 41635877.88358843, 56747440.90702891, 44765761.861980915, 44415067.603860974, 40633608.63996065, 20566349.978868604, 36448289.118144274, 35866893.37906861, 52262573.68648815, 31150361.934648514, 54400206.33363652, 48921485.45446861, 32968168.689456344, 39552414.67396796, 35694313.29435587, 54539130.81081653, 39559766.52902436, 40492267.206768036, 39864026.95534825, 39155683.799664974, 42829564.657392144, 53268963.20053232, 32624213.631804824, 39222113.061419845, 47887162.930692434, 30577280.79355216, 35662879.401265144, 53843222.52352822, 46034

In [3]:
### regression for total cso outflow
y = csoResults["cso"]
print y
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results
reg_m(y,x)
print reg_m(y, x).summary()


[687453499.1268364, 636544508.3291525, 654693027.0751797, 640873155.3622199, 638043088.3812718, 642122196.48792, 649669097.8743469, 632090509.4743917, 635922724.5145442, 635343160.0067642, 659711280.8054883, 648069901.0879678, 647022773.410056, 644583748.3679522, 647651491.6667043, 636596637.8288761, 659871102.8633392, 656283471.6531601, 650499103.4592611, 634264807.1422678, 639129183.5839678, 657673477.1938443, 627108557.2115394, 651121627.8453481, 645817621.243248, 630706058.2198075, 642687737.2648555, 643038431.5229754, 646819890.4868758, 666887149.1479678, 651005210.0086921, 651586605.7477678, 635190925.4403483, 656303137.1921879, 633053292.7931999, 638532013.6723678, 654485330.4373801, 647901084.4528685, 651759185.8324805, 632914368.3160199, 647893732.597812, 646961231.9200684, 647589472.1714882, 648297815.3271714, 644623934.4694443, 634184535.9263041, 654829285.4950316, 648231386.0654166, 639566336.196144, 656876218.3332843, 651790619.7255713, 633610276.6033082, 641418929.613108,

In [4]:
### regression for total volume outflow
y = csoResults["totalVol"]
print y
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results
reg_m(y,x)
print reg_m(y, x).summary()

[1949310793.2803576, 1865779541.2665958, 1895580097.2269776, 1873268597.2879562, 1868247771.7548707, 1874058201.33242, 1887159819.6967874, 1858869437.0984106, 1864153455.0030177, 1863180108.4419215, 1904234826.2228086, 1884350179.0237505, 1883250416.9110167, 1879279472.6352608, 1884035631.3863258, 1866195351.9552126, 1904730241.6130552, 1897910278.4589238, 1888210077.9723136, 1861638184.7608175, 1869248458.868527, 1900735666.3746178, 1849548678.508284, 1890123808.9311955, 1880194455.4312587, 1855930970.6501918, 1875701374.599764, 1876513444.8899138, 1882735861.1142437, 1915292097.0166523, 1888794294.6154778, 1890561358.7589688, 1863556971.8031588, 1898712808.841929, 1860140304.8853645, 1868604875.3225427, 1894540785.944418, 1884953563.003312, 1891490648.0494835, 1859221181.6215467, 1883661689.916198, 1882028211.4027627, 1884671910.204537, 1884909128.71451, 1878327043.8522336, 1861372481.1787217, 1895708848.945012, 1885021507.0703719, 1870095989.067647, 1899522718.00987, 1889853964.8812